In [35]:
import plotly
import chart_studio
import chart_studio.plotly as py
#import plotly.plotly as py
from  plotly.offline import iplot
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()

import pandas as pd
import numpy as np
import glob
import random
from sklearn import preprocessing

# Make sure we are running Python 3
import sys
print(sys.version)

3.6.10 |Anaconda, Inc.| (default, May  7 2020, 19:46:08) [MSC v.1916 64 bit (AMD64)]


# Read & trim data

In [36]:
FOLDER_NAMES = ["tap", "wave", "negative"]
GESTURES = ["tap", "wave", "negative_trim"]

# Manually determined trims (copied from Google Sheets)
# TRIMS = pd.read_csv('trim_ms.csv', header=0, index_col=0, squeeze=True)
# TRIMS = pd.to_numeric(TRIMS, errors='coerce').to_dict()

SEGMENT_LEN_MS = 1500
SAMPLE_FREQ_HZ = 100
SEGMENT_LEN = int(SEGMENT_LEN_MS / SAMPLE_FREQ_HZ * 10) - 5

In [37]:
min_max_scaler = preprocessing.MinMaxScaler()

def read_data(folder_name):
    traces = []
    trace_names = []
    for f in sorted(glob.glob(folder_name + "/*.csv")):
        print(f)
        filename = f[f.find('/') + 1:-4]
        trace = pd.read_csv(f, header=0, index_col="time_ms")
        print(trace.shape)
        trace = trace.apply(pd.to_numeric, errors='coerce')
        trace["accel"] = np.linalg.norm((trace["accel_ms2_x"], trace["accel_ms2_y"], trace["accel_ms2_z"]), axis=0)
        trace["gyro"] = np.linalg.norm((trace['gyro_degs_x'], trace[u'gyro_degs_y'], trace[u'gyro_degs_z']), axis=0)
        
        
        normalized = min_max_scaler.fit_transform(trace['accel_ms2_z'].values.reshape(-1, 1))
        normalized = normalized[:, 0]
        trace["normalized_accel_z"] = normalized
        
        trace_names.append(filename)
        traces.append(trace)
    return trace_names, traces
        
traces = {}
trace_names = {}
for folder_name in FOLDER_NAMES:
    trace_names[folder_name], traces[folder_name] = read_data(folder_name)

tap\tap000.csv
(102, 24)
tap\tap001.csv
(103, 24)
tap\tap002.csv
(102, 24)
tap\tap003.csv
(103, 24)
tap\tap004.csv
(102, 24)
tap\tap005.csv
(102, 24)
tap\tap006.csv
(102, 24)
tap\tap007.csv
(103, 24)
tap\tap008.csv
(102, 24)
tap\tap009.csv
(102, 24)
tap\tap010.csv
(103, 24)
tap\tap011.csv
(103, 24)
tap\tap012.csv
(102, 24)
tap\tap013.csv
(102, 24)
tap\tap014.csv
(102, 24)
tap\tap015.csv
(103, 24)
tap\tap016.csv
(103, 24)
tap\tap017.csv
(103, 24)
tap\tap018.csv
(103, 24)
tap\tap019.csv
(103, 24)
tap\tap020.csv
(100, 24)
tap\tap021.csv
(100, 24)
tap\tap022.csv
(103, 24)
tap\tap023.csv
(103, 24)
tap\tap024.csv
(101, 24)
tap\tap025.csv
(103, 24)
tap\tap026.csv
(103, 24)
tap\tap027.csv
(103, 24)
tap\tap028.csv
(103, 24)
tap\tap029.csv
(103, 24)
tap\tap030.csv
(103, 24)
tap\tap031.csv
(103, 24)
tap\tap032.csv
(102, 24)
tap\tap033.csv
(102, 24)
tap\tap034.csv
(102, 24)
tap\tap035.csv
(102, 24)
tap\tap036.csv
(102, 24)
tap\tap037.csv
(103, 24)
tap\tap038.csv
(102, 24)
tap\tap039.csv
(103, 24)


C:\Users\eryka\miniconda3\envs\my-venv\lib\site-packages\sklearn\preprocessing\data.py:350: RuntimeWarning:

overflow encountered in true_divide

C:\Users\eryka\miniconda3\envs\my-venv\lib\site-packages\sklearn\preprocessing\data.py:370: RuntimeWarning:

invalid value encountered in add



tap\tap041.csv
(103, 24)
tap\tap042.csv
(99, 24)
tap\tap043.csv
(103, 24)
tap\tap044.csv
(102, 24)
tap\tap045.csv
(102, 24)
tap\tap046.csv
(103, 24)
tap\tap047.csv
(103, 24)
tap\tap048.csv
(100, 24)
tap\tap049.csv
(102, 24)
wave\wave000.csv
(102, 24)
wave\wave001.csv
(100, 24)
wave\wave002.csv
(102, 24)
wave\wave003.csv
(103, 24)
wave\wave004.csv
(102, 24)
wave\wave005.csv
(102, 24)
wave\wave006.csv
(102, 24)
wave\wave007.csv
(103, 24)
wave\wave008.csv
(102, 24)
wave\wave009.csv
(103, 24)
wave\wave010.csv
(102, 24)
wave\wave011.csv
(103, 24)
wave\wave012.csv
(103, 24)
wave\wave013.csv
(103, 24)
wave\wave014.csv
(102, 24)
wave\wave015.csv
(100, 24)
wave\wave016.csv
(102, 24)
wave\wave017.csv
(103, 24)
wave\wave018.csv
(102, 24)
wave\wave019.csv
(103, 24)
wave\wave020.csv
(102, 24)
wave\wave021.csv
(103, 24)
wave\wave022.csv
(103, 24)
wave\wave023.csv
(102, 24)
wave\wave024.csv
(102, 24)
wave\wave025.csv
(102, 24)
wave\wave026.csv
(103, 24)
wave\wave027.csv
(102, 24)
wave\wave028.csv
(10

In [38]:
# Split the long negative traces in 1.5 second segments
traces["negative_trim"] = []
trace_names["negative_trim"] = []
for i, trace in enumerate(traces["negative"]):
    for j in range(0, len(trace), 500):
        # Pad an extra 10 to make sure are generous with data points
        traces["negative_trim"].append(trace[j:j + SEGMENT_LEN_MS + 10])
        trace_names["negative_trim"].append(trace_names["negative"][i] + '_' + str(j))

In [39]:
# Trim traces
for gesture in GESTURES:
    for i, trace in enumerate(traces[gesture]):
        filename = trace_names[gesture][i]
        
#         if filename in TRIMS:
#             trim = TRIMS[filename]  
#             if np.isnan(trim) or trim < SEGMENT_LEN_MS:
#                 print('DROPPING TRACE', filename, 'TRIM IS TOO EARLY / BLACKLISTED')
#                 del traces[gesture][i]
#                 del trace_names[gesture][i]
#                 continue
#             trim += random.randrange(0, 200)
#             trace = trace[trim - SEGMENT_LEN_MS:trim]
            
        if len(trace) < SEGMENT_LEN:
            print('DROPPING TRACE', filename, 'NOT ENOUGH DATA POINTS')
            del traces[gesture][i]
            del trace_names[gesture][i]
            continue
            
        traces[gesture][i] = trace.iloc[len(trace) - SEGMENT_LEN:]
        print(filename)
        trace.index = trace.index - trace.index[0]
        traces[gesture][i] = trace

DROPPING TRACE tap\tap000 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap002 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap004 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap006 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap008 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap010 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap012 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap014 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap016 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap018 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap020 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap022 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap024 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap026 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap028 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap030 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap032 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap034 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap036 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\tap038 NOT ENOUGH DATA POINTS
DROPPING TRACE tap\t

# Visualize our data

In [40]:
# List of all sensors
print(traces["tap"][0].columns)

Index(['Unnamed: 0', 'delta_ms', 'accel_ms2_x', 'accel_ms2_y', 'accel_ms2_z',
       'mag_uT_x', 'mag_uT_y', 'mag_uT_z', 'gyro_degs_x', 'gyro_degs_y',
       'gyro_degs_z', 'euler_deg_x', 'euler_deg_y', 'euler_deg_z',
       'quaternion_w', 'quaternion_x', 'quaternion_y', 'quaternion_z',
       'lin_accel_ms2_x', 'lin_accel_ms2_y', 'lin_accel_ms2_z',
       'gravity_ms2_x', 'gravity_ms2_y', 'gravity_ms2_z', 'accel', 'gyro',
       'normalized_accel_z'],
      dtype='object')


In [41]:
SENSORS_LIST = ["accel_ms2_x", "accel_ms2_y", "accel_ms2_z"]
# SENSORS_LIST = ["gyro_degs_x", "gyro_degs_y", "gyro_degs_z"]
# SENSORS_LIST = ["accel"]
SENSORS_LIST = ["normalized_accel_z"]
def plot_all(gesture, large=False):
    for i, trace in enumerate(traces[gesture]):
        data = [] 
        annotations = []
        for sensor in SENSORS_LIST:
            data.append(go.Scatter(
                x = trace.index,
                y = trace[sensor],
                name = sensor,
                line = dict(width = 4 if large else 1)))
            
#             annotations.append(dict(xref='paper', x=1, y=trace[sensor].iloc[-1],
#                                   xanchor='left', yanchor='middle',
#                                   text=sensor,
#                                   showarrow=False))
        layout = go.Layout(
            title = trace_names[gesture][i],
            annotations = annotations,
            font=dict(size=28 if large else 8),
            width=1000,
            margin=go.layout.Margin(r=200, pad=5),
            xaxis = dict(title='Time (ms)'))
            #showlegend=False,
            #yaxis = dict(range=[0, 25])
            #              )
        iplot({'data': data, 'layout': layout}, filename='jupyter-basic_bar')

In [42]:
plot_all("tap")

In [43]:
plot_all("wave")

In [ ]:
plot_all("negative_trim")

# Traditional Machine Learning: Feature Extraction

In [ ]:
import utils

print(utils)

def get_all_features(trace, generate_feature_names=False):
    features = utils.get_model_features(trace, generate_feature_names)
    
    if generate_feature_names:
        features.append('accel_z_peaks')
    else:
        normalized = min_max_scaler.fit_transform(trace['accel_ms2_z'].values.reshape(-1, 1))[:, 0] # normalize
        normalized = normalized[0:len(normalized):5] # subsample
        normalized = np.diff((normalized > 0.77).astype(int)) # convert to binary classifier
        normalized = normalized[normalized>0]
        features.append(sum(normalized))
    
    return features

In [ ]:
# # Sandbox for developing new features

import scipy.fftpack
import matplotlib.pyplot as plot

d = {'tap': [], 'wave': []}
for gesture in d.keys():
    for trace in traces[gesture]:
        normalized = min_max_scaler.fit_transform(trace['accel_ms2_z'].values.reshape(-1, 1))
        normalized = normalized[:, 0]
        normalized = normalized[0:len(normalized):5]
        normalized = np.diff((normalized > 0.77).astype(int))
        normalized = normalized[normalized>0]
        d[gesture].append(sum(normalized))

    viz = pd.Series(d[gesture])
    if gesture is 'tap':
        print('Tap peak count accuracy', float(len(viz[viz == 2])) / len(viz))
    else: 
        print('Wave peak count accuracy', float(len(viz[viz == 3])) / len(viz))
        
    plot.hist(d[gesture],density=1, bins=20) 
    plot.show()

In [ ]:
# Feature Extraction

Y = GESTURES[:]
X = []
FEATURE_NAMES = []
AVG_X = []

for gesture in GESTURES:
    samples = []
    sum_samples = []
    for trace in traces[gesture]:
        if not FEATURE_NAMES:
            FEATURE_NAMES = get_all_features(trace, True)
    
        feature_values = get_all_features(trace)
        
        if not sum_samples:
            sum_samples = feature_values[:]
        else:
            sum_samples = [sum_samples[i] + feature_values[i] for i in range(len(feature_values))]
        samples.append(feature_values)
        
    AVG_X.append([x / float(len(traces[gesture])) for x in sum_samples])
    X.append(samples)

In [ ]:
# Average features for each gesture

average_sample = pd.DataFrame(AVG_X, Y, columns=FEATURE_NAMES).transpose()
display(average_sample)
average_sample.iplot()

# Traditional Machine Learning: Training

In [ ]:
# Put data in SKLearn format and split into train/test datasets

for gesture in GESTURES:
    print(gesture, len(traces[gesture]))
    
from sklearn.model_selection import train_test_split

# Flatten the data
X_flat = []
y_flat = []
for i in range(len(X)):
    X_flat += X[i]
    y_flat += [GESTURES[i]] * len(X[i])
X_flat = np.array(X_flat)
y_flat = np.array(y_flat)

# Generate pretty table of all for display
pretty_table = pd.DataFrame(X_flat, columns=FEATURE_NAMES)
pretty_table['y'] = pd.DataFrame(y_flat)
# display(pretty_table)

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X_flat, y_flat, test_size=0.33, random_state=42)
print(X_train.shape)
# print(y_train)
print(X_test.shape)
# print(y_test)

In [ ]:
# Linear SVC Model

# Actually train the model
from sklearn import svm
from sklearn.metrics import confusion_matrix
model = svm.LinearSVC(max_iter=100000)
model.fit(X_train, y_train) 

# How did we do? 
print("Score:", model.score(X_test, y_test))
predictions = model.predict(X_test)
display(confusion_matrix(y_test, predictions))
df = pd.concat([pd.Series(predictions), pd.Series(y_test)], axis=1)
df.columns=["predicted", "actual"]
display(df)

# Save the model
import joblib
joblib.dump(model, 'models/' + str(len(X_flat)) + 'pt_model.joblib') 

# Set up for deep learning (incomplete)

In [ ]:
from sklearn.model_selection import train_test_split

# Put data in SKLearn format 
traces_flat = []
y_flat = []

# We cap the number of data points in a trace so they are 
# all the same length for our model. Subtract a bit because we are running off
# a non-real-time-system
trace_len_start = 2
trace_len_end = int(SEGMENT_LEN_MS / SAMPLE_FREQ_HZ * 10) - 2

for gesture in GESTURES:
    for trace in traces[gesture]:
        segment = trace[
                ["accel_ms2_x", 
                 "accel_ms2_y", 
                 "accel_ms2_z", 
                 "gyro_degs_x", 
                 "gyro_degs_y", 
                 "gyro_degs_z", 
                 "accel", 
                 "gyro"]]
        traces_flat.append(segment.iloc[trace_len_start:trace_len_end]) 
        y_flat.append(gesture)
    
print("Number of traces: ", len(traces_flat), 
      "\nNumber of features: ", len(traces_flat[0].columns) * len(traces_flat[0]), 
      "\nNumber of sensor streams: ", len(traces_flat[0].columns), 
      "\nNumber of data points per trace", len(traces_flat[0]))

# Split into training and test set
traces_train, traces_test, y_train, y_test = train_test_split(traces_flat, y_flat, test_size=0.33, random_state=42)
print("Number of training traces: ", len(traces_train))
# print(y_train)
print("Number of test traces: ", len(traces_test))
# print(y_test)